In [1]:
# cross-equation coveriance error term
# in each case error terms are gaussian
# what are they picking up
# unobservable errors are correlated

import numpy as np
from matplotlib.pyplot import plot
import pandas as pd
from pathlib import Path
import yfinance as yf

import plotly.graph_objects as go
import plotly.express as px

from os import listdir
from os.path import isfile, join
from datetime import timedelta

local_drive = Path('C:/Users/torin/OneDrive')
databonds = Path('Desktop/Thesis/data/bonds/model')
datamacro = Path('Desktop/Thesis/data/macro')

def normalize_periods(df):
    df['DATE'] = pd.to_datetime(df['DATE'])
    df = df.set_index(['DATE']).resample('MS').mean()
    return df

def calc_difference(df, col_names, new_col_name):
    df[new_col_name] = df[col_names[0]] - df[col_names[1]]
    return df

def calc_ratio(df, col_names, new_col_name):
    df[new_col_name] = df[col_names[0]]/df[col_names[1]]
    return df


def get_yf(ticker):    
    yfpath = Path('C:/Users/torin/OneDrive/Desktop/Thesis/data/macro/yf')
    data = pd.read_csv(yfpath / str(ticker + '.csv'))
    return data

def get_yahoo_finance_series(ticker, name):
    data = yf.Ticker(ticker)
    hist = data.history(period="25y")
    hist = hist.reset_index()
    hist['Date'] = pd.to_datetime(hist['Date'])
    hist.to_csv("C:/Users/torin/OneDrive/Desktop/Thesis/data/macro/yf" +'/' + name + '.csv')
    return hist

tenors=[2,3,4,5,6,7,8,10,12,15,20,25,30]

monhtly_sector_spreads = {}
for tenor in tenors:
    temp = pd.read_csv(local_drive / databonds / Path('IT_BR') / Path('monthly_spreads_' + str(tenor) + '.csv'))
    temp.rename({'date':'DATE'}, axis =1, inplace=True)
    temp = normalize_periods(temp)
    monhtly_sector_spreads[tenor] = temp

yields = pd.concat(monhtly_sector_spreads).reset_index().set_index(['DATE'])

us10y = pd.read_csv(local_drive /  Path('Desktop/Thesis/data') / Path('us10y''.csv'))
us10y['Yield US'] = pd.to_numeric(us10y['IRLTLT01USM156N'])
del us10y['IRLTLT01USM156N']
us10y['tenor'] = 10
us10y = normalize_periods(us10y)

us5y = pd.read_csv(local_drive /  Path('Desktop/Thesis/data') / Path('us5year''.csv'))
us5y['Yield US'] = pd.to_numeric(us5y['DGS5'], errors='coerce')
us5y = us5y.dropna()
del us5y['DGS5']
us5y['tenor'] = 5
us5y = normalize_periods(us5y)


template = 'simple_white'

In [2]:

database = Path('C:/Users/torin/OneDrive/Desktop/Thesis/data')

cds = pd.read_csv(database / Path('macro/model') / Path('cds.csv') )

cds.rename({'Date':'DATE'}, axis = 1, inplace=1)

cds = normalize_periods(cds)

cds = cds.dropna()

cds['BR_US_CDS_Spread'] = cds['BR Price'] - cds['US Price']
cds['IT_US_CDS_Spread'] = cds['IT Price'] - cds['US Price']
cds['BR_IT_CDS_Spread'] = cds['BR Price'] - cds['IT Price']

cdsspreads = cds[['BR_US_CDS_Spread', 'IT_US_CDS_Spread', 'BR_IT_CDS_Spread']]

cdsspreads = cdsspreads/100

cdsspreads

,BR_US_CDS_Spread,IT_US_CDS_Spread,BR_IT_CDS_Spread
DATE,,,
2008-10-01,3.183281,0.506655,2.676626
2008-11-01,3.114040,0.816215,2.297825
2008-12-01,2.883475,1.142000,1.741475
2009-01-01,2.614810,1.043357,1.571452
2009-02-01,2.927637,0.925722,2.001915
...,...,...,...
2023-12-01,0.971119,0.401933,0.569186
2024-01-01,0.948335,0.350096,0.598239
2024-02-01,0.949524,0.368314,0.581210


In [3]:
tenorlist = [5]
data = yields[yields['tenor'].isin(tenorlist)]
y = data[['yield BR']]
data = pd.concat([data, us5y], axis = 1)

data = data.dropna(subset = ['yield BR','Yield US','tenor'])
data['spread_BR_US'] = data['yield BR'] - data['Yield US']
data['spread_IT_US'] = data['yield IT'] - data['Yield US']
data = data[['spread_BR_IT','spread_BR_US','spread_IT_US','yield BR','Yield US','yield IT']]

data = data[data.index.year >= 2000]

data = pd.concat([cdsspreads, data], axis = 1).dropna()

data.to_csv(database / Path('macro/model') / Path('cds_yields_spreads.csv') )

In [6]:
import plotly.graph_objects as go
import plotly.express as px

fig = px.line(data, x=data.index, y=['spread_BR_US',],
              title='Figure 2: Brazil-US 5-year yield spreads (fixed rate bonds).')

fig.update_layout(
    template='simple_white',
    margin=dict(l=0, r=20, t=0, b=60),
     font_family="Times New Roman",
     showlegend=True,
     title_x=0.5, title_y=0.01,
     legend_x = 0.1)

fig.update_layout(
   xaxis_title="",
   yaxis_title="Yield (%)"
)

fig.update_layout(
    autosize=False,
    width=800,
    height=500,
)

fig.update_layout(yaxis_range=[0,15])

fig.update_layout(title={'font': {'size': 20}})

fig.update_xaxes(tickfont=dict(family="Times New Roman",
                                 size=14,) )
fig.update_yaxes(tickfont=dict(family="Times New Roman",
                                 size=14,) )

fig.show()

In [7]:
import plotly.graph_objects as go
import plotly.express as px


fig = px.line(data, x=data.index, y=['IT_US_CDS_Spread', 'BR_US_CDS_Spread'],
              title='CDS Spreads (2010 - 2024)' , template=template )

#fig.add_hline(y=10.487594, line_width=2, line_dash="dash", line_color="black")

fig.update_layout(
    margin=dict(l=0, r=20, t=50, b=0),
    # paper_bgcolor="Black",
     font_family="Calibri",
     showlegend=True,
     title_x=0.47, title_y=0.95,
     legend_x = 0.1)

fig.update_xaxes(
    dtick="M12",
    tickformat="%b\n%Y")

fig.update_layout(
   xaxis_title="",
   yaxis_title=""
)

fig.update_layout(
    autosize=False,
    width=800,
    height=500,
)

fig.update_layout(title={'font': {'size': 16}})

fig.show()

In [8]:
fig = px.line(data, x=data.index, y=['spread_BR_US', 'BR_US_CDS_Spread',],
              title='Figure 4: CDS Spreads (red) and Yield Spreads (blue) between Brazil and U.S., 5-year maturity')

fig.update_layout(
    template='simple_white',
    margin=dict(l=0, r=20, t=0, b=60),
     font_family="Times New Roman",
     showlegend=True,
     title_x=0.5, title_y=0.01,
     legend_x = 0.1)

fig.update_layout(
   xaxis_title="",
   yaxis_title="Yield (%)"
)

fig.update_layout(
    autosize=False,
    width=800,
    height=500,
)

fig.update_layout(yaxis_range=[0,15])

fig.update_layout(title={'font': {'size': 20}})

fig.update_xaxes(tickfont=dict(family="Times New Roman",
                                 size=14,) )
fig.update_yaxes(tickfont=dict(family="Times New Roman",
                                 size=14,) )

fig.show()

In [9]:
pd.DataFrame((data['spread_BR_US'] - data['BR_US_CDS_Spread']).describe())

,0
count,173.000000
mean,6.624881
std,1.824245
min,2.656698
25%,5.214042
50%,7.051867
75%,8.191829
max,9.780983


In [10]:
pd.DataFrame((data['yield BR'] ).describe())

,yield BR
count,173.000000
mean,10.289929
std,2.046398
min,5.360424
25%,9.001368
50%,10.729383
75%,11.627349
max,15.248143


In [11]:
pd.DataFrame((data['spread_IT_US'] - data['IT_US_CDS_Spread']).describe())

,0
count,173.000000
mean,-1.142451
std,1.038475
min,-2.911400
25%,-1.930625
50%,-1.279601
75%,-0.259968
max,1.213488


In [12]:
pd.DataFrame((data['yield IT'] ).describe())

,yield IT
count,173.000000
mean,2.195918
std,1.413195
min,-0.036207
25%,0.908423
50%,2.164950
75%,3.360512
max,5.247877


In [40]:


fig = px.line(data, x=data.index, y=['spread_IT_US', 'IT_US_CDS_Spread'],
              title='CDS Spread vs. Yield Spread, IT-US (2010 - 2024)' , template=template )

#fig.add_hline(y=10.487594, line_width=2, line_dash="dash", line_color="black")

fig.update_layout(
    margin=dict(l=0, r=20, t=50, b=0),
    # paper_bgcolor="Black",
     font_family="Calibri",
     showlegend=True,
     title_x=0.47, title_y=0.95,
     legend_x = 0.1)

fig.update_xaxes(
    dtick="M12",
    tickformat="%b\n%Y")

fig.update_layout(
   xaxis_title="",
   yaxis_title=""
)

fig.update_layout(
    autosize=False,
    width=800,
    height=500,
)

fig.update_layout(title={'font': {'size': 16}})

fig.show()